In [1]:
import torch
import torch.nn as nn
import segmentation_models_pytorch as smp
import time
import cv2
import intel_extension_for_pytorch as ipex

2023-08-13 13:47:54,735 - torch.distributed.nn.jit.instantiator - INFO - Created a temporary directory at /tmp/tmp_l0wbt0m
2023-08-13 13:47:54,737 - torch.distributed.nn.jit.instantiator - INFO - Writing /tmp/tmp_l0wbt0m/_remote_module_non_scriptable.py


[2023-08-13 13:47:54,939] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cpu (auto detect)


2023-08-13 13:48:03.227119: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-13 13:48:03.461218: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
class PetModel(nn.Module):
    def __init__(self, arch, encoder_name, in_channels, out_classes, **kwargs):
        super().__init__()
        self.model = smp.create_model(
            arch, encoder_name=encoder_name, in_channels=in_channels, classes=out_classes, **kwargs
        )

        params = smp.encoders.get_preprocessing_params(encoder_name)
        self.std = torch.tensor(params["std"]).view(1, 3, 1, 1)
        self.mean = torch.tensor(params["mean"]).view(1, 3, 1, 1)
        self.loss_fn = smp.losses.DiceLoss(
            smp.losses.BINARY_MODE, from_logits=True)

    def forward(self, image):
        image = (image - self.mean) / self.std
        mask = self.model(image)
        return mask


In [3]:
model = torch.load("./model/model.pth", map_location=torch.device('cpu'))

In [4]:
# torchscript the model in fp16 mode
model.eval()
example = torch.rand(1, 3, 512, 256).half()
model = torch.jit.trace(model, example)
model = torch.jit.freeze(model)
model = ipex.optimize(model, dtype=torch.float16, level="O3")

/home/venom/miniconda3/lib/python3.9/site-packages/segmentation_models_pytorch/base/model.py:16: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if h % output_stride != 0 or w % output_stride != 0:


In [5]:
def preprocess_frame(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (256, 256))
    frame_org = frame
    frame = torch.from_numpy(frame).float().permute(2, 0, 1).unsqueeze(0)
    return frame

def perform_inference(model, frame):
    inference_start_time = time.time()
    mask = model(frame.half())
    mask = torch.sigmoid(mask)
    mask = mask.float().squeeze().detach().numpy()
    inference_time = time.time() - inference_start_time
    return mask, inference_time

def postprocess_frame(mask):
    # Add any additional post-processing steps here if needed
    post_processing_time = 0  # Placeholder, as there are no explicit post-processing steps
    return post_processing_time

In [6]:
cap = cv2.VideoCapture("./cat.mp4")
while True:
    # Record the start time before processing each frame
    start_time = time.time()
    ret, frame = cap.read()
    if ret:
        # Preprocessing
        preprocessing_start_time = time.time()
        frame = preprocess_frame(frame)
        preprocessing_time = time.time() - preprocessing_start_time
        # Inference
        mask, inference_time = perform_inference(model, frame)
        # Post-processing
        post_processing_time = postprocess_frame(mask)
        # Calculate FPS and frame time
        total_time = time.time() - start_time
        fps = 1.0 / total_time
        frame_time = total_time * 1000.0  # Convert to milliseconds
        # Print FPS and frame time
        print(f"FPS: {fps:.2f}, Frame Time: {frame_time:.2f} ms, "
              f"Preprocessing Time: {preprocessing_time:.2f} s, "
              f"Inference Time: {inference_time:.2f} s, "
              f"Post-processing Time: {post_processing_time:.2f} s", end="\r")
        # Display the processed frame (mask) if you need to
        cv2.imshow('mask', mask)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
    else:
        break